# CUDA STF Tutorial

### Enabled GPU in Colab

### Set up environment

In [2]:
!source ../env.sh

### Check if GPU is running or not, run the following command

In [3]:
!nvidia-smi

Thu May 22 12:39:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.05              Driver Version: 535.86.05    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:1A:00.0 Off |                  N/A |
| 72%   67C    P2             233W / 350W |  18966MiB / 24576MiB |     83%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Check if nvcc compiler is capable of using GPU

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


### `Jacobi example using stream context`

Check <b>[jacobi_pfor.cu](./stf/jacobi_pfor.cu)</b>, compile and run!

In [ ]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/jacobi_pfor.cu -o jacobi_pfor -arch=sm_86 -lcuda --generate-line-info

In [23]:
!./jacobi_pfor

Elapsed time: 1.710080 ms


In [ ]:
!nsys profile --stats=true -t nvtx,cuda,cublas --cuda-event-trace=false --force-overwrite=true -o jacobi_pfor_profile ./jacobi_pfor

### Tasks in the Stream backend

The `stream_ctx` backend utilizes CUDA streams and events to provide synchronization. Each `stream_task` in the `stream_ctx` backend represents a task that is associated with an input CUDA stream. **Asynchronous work can be submitted in the body of the task** using this input stream. Once the `stream_task` completes, all work submitted within the task’s body is assumed to be synchronized with the associated stream.

Question: 提交到一个stream_task中的所有work都是异步执行吗？如果这些work间有依赖呢（嵌套依赖） 

### `AXPY example using graph context`

Check <b>[01-axpy-cuda_kernel_chain.cu](./stf/01-axpy-cuda_kernel_chain.cu)</b>, compile and run!

```cpp
    context ctx = graph_ctx();
```

In [ ]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/01-axpy-cuda_kernel_chain.cu -o 01-axpy-cuda_kernel_chain -arch=sm_86 -lcuda

In [28]:
!./01-axpy-cuda_kernel_chain

In [29]:
!nsys profile --stats=true -t nvtx,cuda,cublas --cuda-event-trace=false --force-overwrite=true -o 01-axpy-cuda_kernel_chain_profile ./01-axpy-cuda_kernel_chain

Generating '/tmp/nsys-report-dfc2.qdstrm'
[1/7] [========================100%] 01-axpy-cuda_kernel_chain_profile.nsys-rep
[2/7] [========================100%] 01-axpy-cuda_kernel_chain_profile.sqlite
[3/7] Executing 'nvtx_sum' stats report
SKIPPED: /root/stf_exp/01-axpy-cuda_kernel_chain_profile.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)   Max (ns)   StdDev (ns)                  Name                
 --------  ---------------  ---------  -----------  -----------  ---------  ---------  -----------  ------------------------------------
     88.6        107030958          1  107030958.0  107030958.0  107030958  107030958          0.0  cudaFree                            
     10.5         12655920          1   12655920.0   12655920.0   12655920   12655920          0.0  cudaGraphLaunch_v10000              
      0.3           413616          3     137872.0    

### `AXPY example using stream context`

Check <b>[01-axpy-cuda_kernel_chain.cu](./stf/01-axpy-cuda_kernel_chain.cu)</b>, compile and run!

```cpp
    context ctx = stream_ctx();
```

In [ ]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/01-axpy-cuda_kernel_chain.cu -o 01-axpy-cuda_kernel_chain_stream -arch=sm_86 -lcuda

In [ ]:
!./01-axpy-cuda_kernel_chain_stream

In [ ]:
!nsys profile --stats=true -t nvtx,cuda,cublas --cuda-event-trace=false --force-overwrite=true -o 01-axpy-cuda_kernel_chain_profile_stream ./01-axpy-cuda_kernel_chain_stream

Generating '/tmp/nsys-report-dfc2.qdstrm'
[1/7] [========================100%] 01-axpy-cuda_kernel_chain_profile.nsys-rep
[2/7] [========================100%] 01-axpy-cuda_kernel_chain_profile.sqlite
[3/7] Executing 'nvtx_sum' stats report
SKIPPED: /root/stf_exp/01-axpy-cuda_kernel_chain_profile.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)   Max (ns)   StdDev (ns)                  Name                
 --------  ---------------  ---------  -----------  -----------  ---------  ---------  -----------  ------------------------------------
     88.6        107030958          1  107030958.0  107030958.0  107030958  107030958          0.0  cudaFree                            
     10.5         12655920          1   12655920.0   12655920.0   12655920   12655920          0.0  cudaGraphLaunch_v10000              
      0.3           413616          3     137872.0    

### `Tasks`

Check <b>[axpy-annotated.cu](./stf/axpy-annotated.cu)</b>, compile and run!

In [6]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/axpy-annotated.cu -o axpy-annotated -arch=sm_86 -lcuda

In [7]:
!CUDASTF_DOT_FILE=axpy.dot ./axpy-annotated

axpy-annotated: stf/axpy-annotated.cu:80: int main(): Assertion `fabs(Y[i] - (Y0(i) + alpha * X0(i))) < 0.0001' failed.


In [5]:
! # Generate the visualization from this dot file in PDF or PNG format
# ! apt install graphviz
! dot -Tpdf axpy.dot -o axpy.pdf
! dot -Tpng axpy.dot -o axpy.png

Check <b>[axpy.pdf](./axpy.pdf)</b>/<b>[axpy.png](./axpy.png)</b>

<img src="./axpy.png" width="30%" height="30%">

### `Synchronization`

Here are simple explanations for CUDASTF synchronization methods:

* `ctx.finalize()` is the main way to wait for all submitted tasks and background operations in the context to complete on the host, while `ctx.submit()` means to initiates the submission of all asynchronous tasks within the sequence.
* `ctx.task_fence()` is used to wait for the completion of all pending operations (tasks, transfers, …).
```cpp
    cudaStream_t stream = ctx.task_fence();
    cudaStreamSynchronize(stream);
```
* `ctx.wait(logical_data)` is a specific blocking method on the host to retrieve the final value of a logical data object, commonly used after reduction tasks.
* Inside kernels launched with `ctx.launch`, `th.sync()` and `th.sync(level)` are used as barriers to synchronize threads at specific levels of the thread hierarchy.

### `Execution Places`

Check <b>[1f1b.cu](./stf/1f1b.cu)</b>, compile and run!

```cpp
    context ctx = stream_ctx();
```

In [15]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/1f1b.cu -o 1f1b -arch=sm_86 -lcuda

In [20]:
!CUDA_VISIBLE_DEVICES=0,1 CUDASTF_DOT_FILE=1f1b.dot ./1f1b

[DEV 0] cannot enable peer access with device 1
[DEV 1] cannot enable peer access with device 0
Number of real devices: 2
../cccl/cudax/include/cuda/experimental/__stf/stream/interfaces/slice.cuh(241) [device 1] CUDA error in data_copy: invalid argument (cudaErrorInvalidValue).


No peer access support in current environment

In [14]:
! # Generate the visualization from this dot file in PDF or PNG format
# ! apt install graphviz
! dot -Tpdf 1f1b.dot -o 1f1b.pdf
! dot -Tpng 1f1b.dot -o 1f1b.png

Check <b>[1f1b.png](./1f1b.png)</b>.

In [ ]:
!nsys profile --stats=true -t nvtx,cuda,cublas --cuda-event-trace=false --force-overwrite=true -o 1f1b_profile ./1f1b

In [ ]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include stf/axpy-annotated.cu -o axpy-annotated -arch=sm_86 -lcuda

In [ ]:
!CUDASTF_DOT_FILE=axpy.dot ./axpy-annotated

axpy-annotated: stf/axpy-annotated.cu:80: int main(): Assertion `fabs(Y[i] - (Y0(i) + alpha * X0(i))) < 0.0001' failed.


In [ ]:
! # Generate the visualization from this dot file in PDF or PNG format
# ! apt install graphviz
! dot -Tpdf axpy.dot -o axpy.pdf
! dot -Tpng axpy.dot -o axpy.png

### `CUDA Thread Hierarchy`

In [ ]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1J_GF6XACL0-Dk1BtJCeWiHwJCFcM_Hkx/edit?usp=share_link&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [13]:
%%writefile mm.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
    for(k = 0; k < size; k++)
       B[i * size + j] += A[i * size + k] * B[k * size + j];

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
       A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
       printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  double t1, t2;

  // Memory Allocation in the Host
  int  *A = (int *) malloc (sizeof(int) * size * size);
  int  *B = (int *) malloc (sizeof(int) * size * size);

  initializeMatrix(A, size);
  initializeMatrix(B, size);

  t1 = omp_get_wtime();

  // Memory Allocation in the Device
  int *d_A, *d_B;
  cudaMalloc((void **) &d_A, size * size * sizeof(int));
  cudaMalloc((void **) &d_B, size * size * sizeof(int));

  // Copy of data from host to device
  cudaMemcpy( d_A, A, size * size * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy( d_B, B, size * size * sizeof(int), cudaMemcpyHostToDevice);

  // 2D Computational Grid
  dim3 dimGrid((int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ));
  dim3 dimBlock( blockSize, blockSize);

       kernel<<<dimGrid, dimBlock>>>(A, B, size);

  // Copy of data from device to host
  cudaMemcpy( B, d_B, size * size * sizeof(int), cudaMemcpyDeviceToHost);

  t2 = omp_get_wtime();

  printf("%d\t%f\n", size, t2-t1);

 //printMatrix(B, size);

 // Memory Allocation in the Device
 cudaFree(d_A);
 cudaFree(d_B);

 // Memory Allocation in the Host
 free(A);
 free(B);

 return 0;

}

Writing mm.cu


In [14]:
!nvcc -arch=sm_75 mm.cu -o mm -Xcompiler -fopenmp

In [9]:
!./mm 1000 64

1000	0.239031


### `Grid-Stride Loops`

In [ ]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1tRO-HwqCfv8imhDO4S_8yAv8wEcJVttZ/edit?usp=sharing&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [15]:
%%writefile mm-gridStrideLoop.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
    for(k = 0; k < size; k++)
       B[i * size + j] += A[i * size + k] * B[k * size + j];

}

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
            B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  double t1, t2;

  // Memory Allocation in the Host
  int  *A = (int *) malloc (sizeof(int) * size * size);
  int  *B = (int *) malloc (sizeof(int) * size * size);

  initializeMatrix(A, size);
  initializeMatrix(B, size);

  t1 = omp_get_wtime();

  // Memory Allocation in the Device
  int *d_A, *d_B;
  cudaMalloc((void **) &d_A, size * size * sizeof(int));
  cudaMalloc((void **) &d_B, size * size * sizeof(int));

  // Copy of data from host to device
  cudaMemcpy( d_A, A, size * size * sizeof(int), cudaMemcpyHostToDevice );
  cudaMemcpy( d_B, B, size * size * sizeof(int), cudaMemcpyHostToDevice );

  // 2D Computational Grid
  dim3 dimGrid( (int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ) );
  dim3 dimBlock( blockSize, blockSize);

            kernelGridStrideLoop<<<dimGrid, dimBlock>>>(A, B, size);

  // Copy of data from device to host
  cudaMemcpy( B, d_B, size * size * sizeof(int), cudaMemcpyDeviceToHost );

  t2 = omp_get_wtime();

  printf("%d\t%f\n", size, t2-t1);

 //printMatrix(A, size);
 //printMatrix(B, size);

 // Memory Allocation in the Device
 cudaFree(d_A);
 cudaFree(d_B);

 // Memory Allocation in the Host
 free(A);
 free(B);

 return 0;
}

Writing mm-gridStrideLoop.cu


In [16]:
!nvcc -arch=sm_75 mm-gridStrideLoop.cu -o mm-gridStrideLoop -Xcompiler -fopenmp

In [12]:
!./mm-gridStrideLoop 1000 64

1000	0.136449


### `Unified Memory (cudaMallocManaged)`

In [ ]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1ZgEGCivfxKS6DDHsq1-3-k4YELQQknZ0/edit?usp=share_link&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [17]:
%%writefile mm-cudaMallocManaged.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
     for(k = 0; k < size; k++)
        B[i * size + j] += A[i * size + k] * B[k * size + j];

}

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
            B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
 int size = atoi(argv[1]);
 int blockSize = atoi(argv[2]); ;
 double t1, t2;
 int *A,  *B;

 t1 = omp_get_wtime();

 cudaMallocManaged(&A, sizeof(int) * size * size);
 cudaMallocManaged(&B, sizeof(int) * size * size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 dim3 dimGrid( (int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ) );
 dim3 dimBlock( blockSize, blockSize);

      kernelGridStrideLoop<<<dimGrid, dimBlock>>>(A, B, size);
      cudaDeviceSynchronize();

 t2 = omp_get_wtime();

printf("%d\t%f\n", size, (t2-t1));

//printMatrix(A, size);
//printMatrix(B, size);

// Free all our allocated memory
cudaFree(A);
cudaFree(B);

return 0;

}


Writing mm-cudaMallocManaged.cu


In [18]:
!nvcc mm-cudaMallocManaged.cu -o mm-cudaMallocManaged -Xcompiler -fopenmp

In [15]:
!./mm-cudaMallocManaged 1000 64

1000	0.177393


#### `Streaming Multiprocessors (SMs)`

In [ ]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/18z3x55kxCCjGZ3LVKOtSN5q8qXe4swFL/edit?usp=sharing&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [19]:
%%writefile mm-streamingMultiprocessors.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
         B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 int size = atoi(argv[1]);
 int sizeblock = atoi(argv[2]); ;
 double t1, t2;
 int *A,  *B;

 t1 = omp_get_wtime();

 cudaMallocManaged(&A, sizeof(int) * size * size);
 cudaMallocManaged(&B, sizeof(int) * size * size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 int deviceId, numberOfSMs;
 cudaGetDevice(&deviceId);
 cudaDeviceGetAttribute(&numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId);

 int NUMBER_OF_BLOCKS = numberOfSMs * 32;
 int NUMBER_OF_THREADS = 1024;

      kernelGridStrideLoop<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS>>>(A, B, size);
      cudaDeviceSynchronize();

 t2 = omp_get_wtime();

 printf("%d\t%f\n", size, t2-t1);

//printMatrix(B, size);

// Free all our allocated memory
 cudaFree(A);
 cudaFree(B);

 return 0;
}

Writing mm-streamingMultiprocessors.cu


In [20]:
!nvcc mm-streamingMultiprocessors.cu -o mm-streamingMultiprocessors -Xcompiler -fopenmp

In [18]:
!./mm-streamingMultiprocessors 1000 64

1000	0.174564


## `Profilling GPU core`

The GPU has many units working in parallel, and it is common for it to be bound by different units at different frame sequences. Due to the nature of this behavior, it is beneficial to identify where the GPU cost is going when searching for bottlenecks and to understand what a GPU bottleneck is. Some applications help developers identify bottlenecks, which is useful when optimizing performance, following some NVIDIA profilling tools.

In [3]:
%%writefile vector-add.cu
#include <stdio.h>
#include <cuda.h>

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
    a[i] = num;

}

__global__ void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
    result[i] = a[i] + b[i];
}

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main(int argc, char **argv)
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  int deviceId;
  cudaGetDevice(&deviceId);

  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);
  int multiProcessorCount = props.multiProcessorCount;
  threadsPerBlock = 1024;
  numberOfBlocks = 32 * multiProcessorCount;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  // Free all our allocated memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

Writing vector-add.cu


In [6]:
!nvcc vector-add.cu -o vector-add

### ⊗ NSYS

`NVIDIA Nsight Systems` (nsys) is a system-wide performance analysis tool designed to visualize an application’s algorithms, help you identify the largest opportunities to optimize, and tune to scale efficiently across any quantity or size of GPUs.

The command `nsys profile` will generate a `qdrep` report file which can be used in a variety of manners. We use the `--stats=true` flag here to indicate we would like summary statistics printed. There is quite a lot of information printed:

- Profile configuration details
- Report file(s) generation details
- **CUDA API Statistics**
- **CUDA Kernel Statistics**
- **CUDA Memory Operation Statistics (time and size)**
- OS Runtime API Statistics

In this lab you will primarily be using the nsys im command line. In the next, you will be using the generated report files to give to the Nsight Systems GUI for visual profilling.

In [7]:
!nsys profile --stats=true ./vector-add

Error: the provided PTX was compiled with an unsupported toolchain.
FAIL: vector[0] - 0 does not equal 7
Generating '/tmp/nsys-report-150b.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     95.9    1,220,708,427         56  21,798,364.8  10,077,186.5     1,245  316,726,757  47,004,542.7  poll                  
      3.8       48,384,557        490      98,744.0      11,310.5     1,035   16,905,786     828,425.6  ioctl                 
      0.2        2,008,964         27      74,406.1      14,791.0    10,641 

After profilling the application, answer the following questions using information displayed in the `CUDA Kernel Statistics` section of the profilling output.

## `Concurrent CUDA Streams`

In CUDA programming, a **stream** is a series of commands that execute in order. In CUDA applications, kernel execution, as well as some memory transfers, occur within CUDA streams. Up until this point in time, you have not been interacting explicitly with CUDA streams, but in fact, your CUDA code has been executing its kernels inside of a stream called *the default stream*. CUDA programmers can create and utilize non-default CUDA streams in addition to the default stream, and in doing so, perform multiple operations, such as executing multiple kernels, concurrently, in different streams. Using multiple streams can add an additional layer of parallelization to your accelerated applications, and offers many more opportunities for application optimization.

### Creating, Utilizing, and Destroying Non-Default CUDA Streams

The following code snippet demonstrates how to create, utilize, and destroy a non-default CUDA stream. You will note, that to launch a CUDA kernel in a non-default CUDA stream, the stream must be passed as the optional 4th argument of the execution configuration. Up until now you have only utilized the first 2 arguments of the execution configuration:

```cpp
cudaStream_t stream;       // CUDA streams are of type `cudaStream_t`.
cudaStreamCreate(&stream); // Note that a pointer must be passed to `cudaCreateStream`.

someKernel<<<number_of_blocks, threads_per_block, 0, stream>>>(); // `stream` is passed as 4th EC argument.

cudaStreamDestroy(stream); // Note that a value, not a pointer, is passed to `cudaDestroyStream`.
```

Outside the scope of this lab, but worth mentioning, is the optional 3rd argument of the execution configuration. This argument allows programmers to supply the number of bytes in **shared memory** (an advanced topic that will not be covered presently) to be dynamically allocated per block for this kernel launch. The default number of bytes allocated to shared memory per block is `0`, and for the remainder of the lab, you will be passing `0` as this value, in order to expose the 4th argument.

In [21]:
%%writefile print-numbers-solution.cu
#include <stdio.h>
#include <unistd.h>

__global__ void printNumber(int number)
{
  printf("%d\n", number);
}

int main(int argc, char **argv)
{
  int i;

  for(i = 0; i < 5; ++i)
  {
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    printNumber<<<1, 1, 0, stream>>>(i);
    cudaStreamDestroy(stream);
  }

  cudaDeviceSynchronize();

  return 0;
}

Writing print-numbers-solution.cu


In [22]:
!nvcc print-numbers-solution.cu -o print-numbers-solution

In [ ]:
!./print-numbers-solution

0
1
2
3
4


## `Asynchronous Memory Prefetching`

Prefetching also tends to migrate data in larger chunks, and therefore fewer trips, than on-demand migration. This makes it an excellent fit when data access needs are known before runtime, and when data access patterns are not sparse.

CUDA Makes asynchronously prefetching managed memory to either a GPU device or the CPU easy with its `cudaMemPrefetchAsync` function. Here is an example of using it to both prefetch data to the currently active GPU device, and then, to the CPU:

```cpp

int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.

```

In [23]:
%%writefile vector-add-prefetching.cu
#include <stdio.h>
#define N 2048 * 2048 // Number of elements in each vector

__global__ void saxpy(int * a, int * b, int * c)
{
  // Determine our unique global thread ID, so we know which element to process
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for (int i = tid; i < N; i += stride)
     c[i] = 2 * a[i] + b[i];
}

int main(int argc, char **argv)
{
  int *a, *b, *c;

  int size = N * sizeof (int); // The total number of bytes per vector

  int deviceId;
  int numberOfSMs;

  cudaGetDevice(&deviceId);
  cudaDeviceGetAttribute(&numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId);

  // Allocate memory
  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  // Initialize memory
  for(int i = 0; i < N; ++i )
  {
    a[i] = 2;
    b[i] = 1;
    c[i] = 0;
  }

  cudaMemPrefetchAsync(a, size, deviceId);
  cudaMemPrefetchAsync(b, size, deviceId);
  cudaMemPrefetchAsync(c, size, deviceId);

  int threads_per_block = 256;
  int number_of_blocks = numberOfSMs * 32;

  saxpy <<<number_of_blocks, threads_per_block>>>( a, b, c );

  cudaDeviceSynchronize(); // Wait for the GPU to finish

  // Print out the first and last 5 values of c for a quality check
  for( int i = 0; i < 5; ++i )
    printf("c[%d] = %d, ", i, c[i]);
  printf ("\n");
  for( int i = N-5; i < N; ++i )
    printf("c[%d] = %d, ", i, c[i]);
  printf ("\n");

  // Free all our allocated memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

Writing vector-add-prefetching.cu


In [24]:
!nvcc vector-add-prefetching.cu -o vector-add-prefetching

In [ ]:
!./vector-add-prefetching

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
